In [33]:
import pandas as pd
import fasttext
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import matplotlib.pyplot as plt
import csv
import ast
import re

In [2]:
# get the df with the posts, tokenized posts, birth years, and binary birth years
df_birth_year = pd.read_csv('data/birth_year.csv')
mil_and_genz = df_birth_year[(1986 < df_birth_year['birth_year']) & (df_birth_year['birth_year'] <= 2006)]
mil_and_genz['binary_birth_year'] = 1
mil_and_genz.loc[(1996 < mil_and_genz['birth_year']) & (mil_and_genz['birth_year'] <= 2006), 'binary_birth_year'] = 0
mil_and_genz = mil_and_genz.reset_index(drop=True)
mil_and_genz

# underample the minority class
undersampled_df = pd.concat([
    resample(mil_and_genz[mil_and_genz['binary_birth_year'] == 1],
             replace=False,
             n_samples=len(mil_and_genz[mil_and_genz['binary_birth_year'] == 0]),
             random_state=42),
    mil_and_genz[mil_and_genz['binary_birth_year'] == 0]
])

# shuffle the dataframe
undersampled_df = undersampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

undersampled_df

# tokenization
mil_and_genz['post_tokenized'] = mil_and_genz.post.str.findall('\w+|[^\w\s]')
mil_and_genz

<ipython-input-2-e67fa423ed4e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mil_and_genz['binary_birth_year'] = 1
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,auhtor_ID,post,birth_year,binary_birth_year,post_tokenized
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001,0,"[Good, on, you, for, being, responsible, !, I,..."
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001,0,"[must, go, to, the, grocery, store, with, thei..."
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001,0,"[things, on, her, videos, ,, and, YouTube, too..."
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001,0,"[their, app, ., There, ', s, also, a, program,..."
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001,0,"[side, ., If, the, cops, don, ', t, take, your..."
...,...,...,...,...,...
25450,t2_ef4dv976,clever political meme on a sub about political...,1989,1,"[clever, political, meme, on, a, sub, about, p..."
25451,t2_7ps3a,Kevin. Glück gehabt. I will try! Thank you! I ...,1992,1,"[Kevin, ., Glück, gehabt, ., I, will, try, !, ..."
25452,t2_7ps3a,Handy/der Cloud speichern und in der Kamera ha...,1992,1,"[Handy, /, der, Cloud, speichern, und, in, der..."
25453,t2_7ps3a,that. :)) On Google I cant find much informati...,1992,1,"[that, ., :, ), ), On, Google, I, cant, find, ..."


In [3]:
# concatenate all posts into a single string and save
corpus = " ".join([" ".join(post) for post in mil_and_genz['post_tokenized']])
with open("data/temp_corpus.txt", "w", encoding="utf-8") as file:
    file.write(corpus)

# fastText word embeddings
model = fasttext.train_unsupervised("data/temp_corpus.txt")
model.save_model("result/trained_fasttext_embeddings.bin")

In [4]:
# load the trained model
model = fasttext.load_model("result/trained_fasttext_embeddings.bin")

# get the average embedding for a post (document embedding)
def get_average_embedding(tokens, model):
    embeddings = [model.get_word_vector(word) for word in tokens if word in model.words]
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
    else:
        avg_embedding = np.zeros(model.get_dimension())  # Default to zero if no embeddings found
    return avg_embedding

mil_and_genz['doc_embedding'] = mil_and_genz['post_tokenized'].apply(lambda x: get_average_embedding(x, model))

# averaging the document embeddings

In [36]:
def calculate_average(lst):
    return sum(lst) / len(lst) if len(lst) > 0 else None

# Apply the function to the specified column and create a new column with averages
mil_and_genz['doc_embedding_average'] = mil_and_genz['doc_embedding'].apply(calculate_average)
mil_and_genz.to_csv('data/mil_and_genz.csv', index=False)

In [37]:
mil_and_genz

,auhtor_ID,post,birth_year,binary_birth_year,post_tokenized,doc_embedding,doc_embedding_average,doc_length
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001,0,"[Good, on, you, for, being, responsible, !, I,...","[0.077345595, -0.14746565, 0.26790008, 0.31544...",0.017216,1550
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001,0,"[must, go, to, the, grocery, store, with, thei...","[0.057216443, -0.12836617, 0.25229907, 0.32476...",0.016122,1560
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001,0,"[things, on, her, videos, ,, and, YouTube, too...","[0.06751289, -0.11984211, 0.2425011, 0.3101618...",0.016001,1542
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001,0,"[their, app, ., There, ', s, also, a, program,...","[0.059638795, -0.114736974, 0.23432401, 0.3349...",0.016488,1567
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001,0,"[side, ., If, the, cops, don, ', t, take, your...","[0.06526452, -0.09764939, 0.2473862, 0.3262983...",0.016004,1552
...,...,...,...,...,...,...,...,...
25450,t2_ef4dv976,clever political meme on a sub about political...,1989,1,"[clever, political, meme, on, a, sub, about, p...","[0.073144875, -0.124933265, 0.2885858, 0.30519...",0.017437,1074
25451,t2_7ps3a,Kevin. Glück gehabt. I will try! Thank you! I ...,1992,1,"[Kevin, ., Glück, gehabt, ., I, will, try, !, ...","[0.09270658, 0.069318876, 0.06377351, 0.368193...",0.021317,1484
25452,t2_7ps3a,Handy/der Cloud speichern und in der Kamera ha...,1992,1,"[Handy, /, der, Cloud, speichern, und, in, der...","[0.08413791, 0.08132758, 0.046772774, 0.379053...",0.022058,1490
25453,t2_7ps3a,that. :)) On Google I cant find much informati...,1992,1,"[that, ., :, ), ), On, Google, I, cant, find, ...","[0.11956645, 0.18445647, -0.063145295, 0.37309...",0.027131,1504


# adding doc length, should be removed later

In [11]:
# document length
def count_words(tokens):
    return sum(1 for token in tokens if token.isalpha())
mil_and_genz['doc_length'] = mil_and_genz['post_tokenized'].apply(count_words)
mil_and_genz

,auhtor_ID,post,birth_year,binary_birth_year,post_tokenized,doc_embedding,doc_embedding_average,doc_length
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001,0,"[Good, on, you, for, being, responsible, !, I,...","[0.077345595, -0.14746565, 0.26790008, 0.31544...",0.017216,1550
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001,0,"[must, go, to, the, grocery, store, with, thei...","[0.057216443, -0.12836617, 0.25229907, 0.32476...",0.016122,1560
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001,0,"[things, on, her, videos, ,, and, YouTube, too...","[0.06751289, -0.11984211, 0.2425011, 0.3101618...",0.016001,1542
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001,0,"[their, app, ., There, ', s, also, a, program,...","[0.059638795, -0.114736974, 0.23432401, 0.3349...",0.016488,1567
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001,0,"[side, ., If, the, cops, don, ', t, take, your...","[0.06526452, -0.09764939, 0.2473862, 0.3262983...",0.016004,1552
...,...,...,...,...,...,...,...,...
25450,t2_ef4dv976,clever political meme on a sub about political...,1989,1,"[clever, political, meme, on, a, sub, about, p...","[0.073144875, -0.124933265, 0.2885858, 0.30519...",0.017437,1074
25451,t2_7ps3a,Kevin. Glück gehabt. I will try! Thank you! I ...,1992,1,"[Kevin, ., Glück, gehabt, ., I, will, try, !, ...","[0.09270658, 0.069318876, 0.06377351, 0.368193...",0.021317,1484
25452,t2_7ps3a,Handy/der Cloud speichern und in der Kamera ha...,1992,1,"[Handy, /, der, Cloud, speichern, und, in, der...","[0.08413791, 0.08132758, 0.046772774, 0.379053...",0.022058,1490
25453,t2_7ps3a,that. :)) On Google I cant find much informati...,1992,1,"[that, ., :, ), ), On, Google, I, cant, find, ...","[0.11956645, 0.18445647, -0.063145295, 0.37309...",0.027131,1504
